- La resta de fondo (BS) es una técnica común y ampliamente utilizada para generar una máscara de primer plano (es decir, una imagen binaria que contiene los píxeles que pertenecen a objetos en movimiento en la escena) mediante el uso de cámaras estáticas.
- Como su nombre indica, BS calcula la máscara de primer plano realizando una resta entre el fotograma actual y un modelo de fondo, que contiene la parte estática de la escena o, más en general, todo lo que puede considerarse como fondo dadas las características de la escena observada.

El modelado en segundo plano consta de dos pasos principales:

- Inicialización de fondo;
- Actualización de antecedentes.

En el primer paso se calcula un modelo inicial del fondo, mientras que en el segundo paso ese modelo se actualiza para adaptarse a posibles cambios en la escena.

In [ ]:
from __future__ import print_function
import cv2 as cv
import argparse

parser = argparse.ArgumentParser(description='This program shows how to use background subtraction methods provided by \
                                              OpenCV. You can process both videos and images.')
parser.add_argument('--input', type=str, help='Path to a video or a sequence of image.', default='resources/vtest.avi')
parser.add_argument('--algo', type=str, help='Background subtraction method (KNN, MOG2).', default='MOG2')
args = parser.parse_args()

## [create]
#Se utilizará un objeto BackgroundSubtractor() para generar la máscara de primer plano.
#En este ejemplo, se utilizan parámetros predeterminados, pero también es posible declarar parámetros específicos en la función de creación.
if args.algo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
## [create]

#Un objeto VideoCapture() se utiliza para leer la secuencia de vídeo de entrada o de imágenes de entrada.
capture = cv.VideoCapture(cv.samples.findFileOrKeep(args.input))
if not capture.isOpened():
    print('Unable to open: ' + args.input)
    exit(0)
## [capture]

while True:
    ret, frame = capture.read()
    if frame is None:
        break

    ## [apply]
    #Cada fotograma se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    #Si desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo en segundo plano, es posible establecer una tasa de aprendizaje específica pasando un parámetro al applymétodo.
    fgMask = backSub.apply(frame)
    ## [apply]

    ## [display_frame_number]
    #El número de fotograma actual se puede extraer del objeto VideoCapture y estampar en la esquina superior izquierda del fotograma actual. Se utiliza un rectángulo blanco para resaltar el número de fotograma de color negro.
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    ## [display_frame_number]

    ## [show]
    #Estamos listos para mostrar el cuadro de entrada actual y los resultados.
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    ## [show]

    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break